In [1]:
import csv
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
def get_geometry_vector(file_id):
	fo=open(file_id,'r')
	vector=[]
	data=fo.readlines()
	for i in range(2,len(data)):
		elements=data[i].split()
		if elements[0]=='C':
			vector.extend([1,0,0,0])
		elif elements[0]=='N':
			vector.extend([0,1,0,0])
		elif elements[0]=='O':
			vector.extend([0,0,1,0])
		elif elements[0]=='H':
			vector.extend([0,0,0,1])
		for j in range(1,len(elements)):
			vector.append(float(elements[j]))
	return vector

In [3]:
fo=open('./train.csv')
data=csv.reader(fo)
d={}
count=0
maxx=count
for i in data:
	for j in i[1]:
		if j not in d:
			d[j]=count
			count+=1

In [4]:
fo=open('./test.csv')
data=csv.reader(fo)
d={}
for i in data:
# 	print('loop1')
	for j in i[1]:
		if j not in d:
			d[j]=count
			count+=1

In [5]:
X_train=[]
Y_train=[]
# print(d)
fo.close()
fo=open('./train.csv')
data=csv.reader(fo)
num=0
maxlen=0
for i in data:
	if num==0:
		num=1
		continue
	#print(float(i[2]))
	Y_train.append(float(i[2]))
	cur=[]
	for j in i[1]:
		val=d[j]
		one_hot=[0 for i in range(count)]
		one_hot[val]=1
		cur.extend(one_hot)
	maxlen=max(maxlen,len(cur))
	X_train.append(cur)

In [6]:
X_final=[]
Y_final=[]

fo=open('./test.csv')
data=csv.reader(fo)
num=0
maxlen=0
for i in data:
	if num==0:
		num=1
		continue
	#print(float(i[2]))
	#Y_train.append(float(i[2]))
	cur=[]
	for j in i[1]:
		val=d[j]
		one_hot=[0 for i in range(count)]
		one_hot[val]=1
		cur.extend(one_hot)
	maxlen=max(maxlen,len(cur))
	X_final.append(cur)

for i in range(len(X_train)):
	for j in range(maxlen-len(X_train[i])):
		X_train[i].append(0)
# 	print(len(X_train[i]))
print(len(X_train))

for i in range(len(X_final)):
	for j in range(maxlen-len(X_final[i])):
		X_final[i].append(0)
# 	print(len(X_final[i]))
print(len(X_final))

10000
3000


In [7]:
num=0
fo.close()
fo=open('./train.csv')
data=csv.reader(fo)
counter=0
for i in data:
	if num==0:
		num=1
		continue
	else:
		num+=1
	file_id='./xyzs/'+i[0]
	vector=get_geometry_vector(file_id)
# 	print(vector)
	X_train[counter].extend(get_geometry_vector(file_id))
	maxlen=max(maxlen,len(X_train[counter]))
	counter+=1

In [8]:
num=0
fo.close()
fo=open('./test.csv')
data=csv.reader(fo)
counter=0
for i in data:
	if num==0:
		num=1
		continue
	else:
		num+=1
	file_id='./xyzs/'+i[0]
	vector=get_geometry_vector(file_id)
# 	print(vector)
	X_final[counter].extend(get_geometry_vector(file_id))
	maxlen=max(maxlen,len(X_final[counter]))
	counter+=1

for i in range(len(X_train)):
	for j in range(maxlen-len(X_train[i])):
		X_train[i].append(0)
	#print(len(X_train[i]))

for i in range(len(X_final)):
	for j in range(maxlen-len(X_final[i])):
		X_final[i].append(0)

In [9]:
X_train=np.array(X_train)
print(X_train.shape)
print(len(Y_train))
X_final=np.array(X_final)
print(X_final.shape)
X_train,X_test,Y_train,Y_test=train_test_split(X_train,Y_train, test_size=0.2, random_state=42)
#print("debug")
#print(Y_train)
print(X_train[0])
params = {'n_estimators':100, 'max_depth': 3, 'min_samples_split': 2,
          'learning_rate': 0.1, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf.fit(X_train,Y_train)
# preds=clf.predict(X_final)
# np.savetxt('out.txt',preds)

(10000, 2043)
10000
(3000, 2043)
[0. 0. 0. ... 0. 0. 0.]


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [10]:
preds = clf.predict(X_test)
import math
import sklearn

rmse = math.sqrt(sklearn.metrics.mean_squared_error(Y_test, preds))
print(rmse)

30.793733056340933


In [12]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(Y_test, preds)

18.877410349513664

In [15]:
avg_energy = sum(Y_test)/len(Y_test)
avg_preds = [avg_energy for x in Y_test]

In [18]:
mean_absolute_error(Y_test, avg_preds)

187.75382368399795

In [19]:
math.sqrt(sklearn.metrics.mean_squared_error(Y_test, avg_preds))

239.45844336270324